# Systematics Notebook
From results of the phi fitting.

In [1]:
import glob 
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import pickle
import sys
import time 

%matplotlib inline

plt.rc('font', family='serif')
plt.rc('font', size=18)

### Load Configurations
There are several files with different results for phi-distributions.  

In [2]:
database_files = glob.glob('database/fit/*.csv')
print('Found %d files in the database.' % len(database_files))

Found 23 files in the database.


In [3]:
def load_database_files(file_list):
    
    dataframe_store = {}
    for f in file_list:
        dataframe_store[f] = pd.read_csv(f)
        
    return dataframe_store

In [4]:
dataframe_store = load_database_files(database_files)

Finally, fit the nominal phi distributions.

In [5]:
nominal = pd.read_csv('database/fit/sys.csv')

### Systematic Uncertainties 

In [6]:
def database_filename_parser(file_name):
    file_name = file_name.split('variation_')[-1].strip('.csv')
    
    tokens = file_name.split('_')
    n_tokens = len(tokens)

    parameter_name = '_'.join(tokens[0:n_tokens-1])
    index = int(tokens[-1])
    
    return index, parameter_name

In [7]:
def build_parameter_variation_dict(path_to_db = 'database/fit/'):

    parameters = {}

    database_files = glob.glob(path_to_db + 'variation*.csv')
    
    for database_file in database_files:
        index, parameter = database_filename_parser(database_file)
        
        if parameter in parameters.keys():
            parameters[parameter][index] = pd.read_csv(database_file)
        else:
            parameters[parameter] = {}
            parameters[parameter][index] = pd.read_csv(database_file)
            
    return parameters

In [8]:
def load_systematic_sources_list(file_name):
    systematic_sources = pickle.load(open(file_name, 'rb'))
    
    reverse_dict = {}

    for key, value in systematic_sources.iteritems():
        reverse_dict[value] = key
    
    return reverse_dict

In [9]:
parameter_variation = build_parameter_variation_dict()

In [10]:
for key, value in parameter_variation.iteritems():
    print(key, value.keys())

('dist_dcr3', [1, -1])
('dist_dcr1', [1, -1])
('dist_vz', [1, -1])
('dist_ecsf', [1, -1])
('dist_ecw', [1, -1])
('dist_ec_edep', [1, -1])
('alpha', [1, -1])
('p_mes', [0, -1])
('dist_ecu', [1, -1])
('dist_cc', [0, -1])
('dist_ecv', [1, -1])


In [11]:
systematic_sources = load_systematic_sources_list('systematic_sources.pkl')

In [12]:
def add_systematics(nominal_fit, parameter_variation, systematic_sources):

    nominal_fit_sys = nominal_fit.copy(deep=True)
    nominal_fit_sys['sys_total_0'] = np.zeros(len(nominal_fit_sys))
    nominal_fit_sys['sys_total_1'] = np.zeros(len(nominal_fit_sys))
    nominal_fit_sys['sys_total_2'] = np.zeros(len(nominal_fit_sys))
    
    for key in parameter_variation.keys():

        min_index = parameter_variation[key].keys()[0]    
        max_index = parameter_variation[key].keys()[-1]

        merged_data = pd.merge(parameter_variation[key][min_index], 
                 parameter_variation[key][max_index],
                 on = ['axis', 'axis_bin'])

        merged_data[systematic_sources[key] + '_par0'] = np.abs(merged_data.par_0_y - merged_data.par_0_x)
        merged_data[systematic_sources[key] + '_par1'] = np.abs(merged_data.par_1_y - merged_data.par_1_x)
        merged_data[systematic_sources[key] + '_par2'] = np.abs(merged_data.par_2_y - merged_data.par_2_x)

        merge_cols = ['axis', 'axis_bin', systematic_sources[key] + '_par0', 
                     systematic_sources[key] + '_par1', systematic_sources[key] + '_par2']

        nominal_fit_sys = pd.merge(nominal_fit_sys, merged_data[merge_cols], 
                                   on = ['axis', 'axis_bin'])
    
        nominal_fit_sys.sys_total_0 += nominal_fit_sys[systematic_sources[key] + '_par0']**2
        nominal_fit_sys.sys_total_1 += nominal_fit_sys[systematic_sources[key] + '_par1']**2
        nominal_fit_sys.sys_total_2 += nominal_fit_sys[systematic_sources[key] + '_par2']**2


    nominal_fit_sys.sys_total_0 = np.sqrt(nominal_fit_sys.sys_total_0)
    nominal_fit_sys.sys_total_1 = np.sqrt(nominal_fit_sys.sys_total_1)
    nominal_fit_sys.sys_total_2 = np.sqrt(nominal_fit_sys.sys_total_2)

    
    return nominal_fit_sys

In [13]:
nominal_fit_sys = add_systematics(nominal, parameter_variation, systematic_sources)

In [14]:
nominal_fit_sys.to_csv('results/fit/sys.csv', index=False)

In [15]:
nominal_fit_sys.head(24)

,axis,axis_bin,axis_max,axis_min,err_0,err_1,err_2,par_0,par_1,par_2,...,sys_7_par2,sys_3_par0,sys_3_par1,sys_3_par2,sys_5_par0,sys_5_par1,sys_5_par2,sys_11_par0,sys_11_par1,sys_11_par2
0,pt,0,0.202108,0.000284,0.005593,0.399494,0.481435,0.023548,0.035530,-0.284411,...,0.039661,0.000101,0.133262,0.118209,0.000456,0.061791,0.034219,0.000964,0.026559,0.005895
1,pt,1,0.274544,0.202108,0.006052,0.395408,0.481255,0.025324,-0.422032,0.095175,...,0.040875,0.001937,0.007376,0.167376,0.001371,0.062128,0.016032,0.001149,0.041276,0.014100
2,pt,2,0.328453,0.274544,0.006538,0.372410,0.388941,0.030275,0.148326,-0.380049,...,0.094868,0.001548,0.091878,0.010678,0.001482,0.047843,0.015970,0.000358,0.011726,0.054564
3,pt,3,0.376169,0.328453,0.004896,0.414820,0.458937,0.033449,-0.043825,-0.252222,...,0.011658,0.000245,0.025403,0.004015,0.001216,0.036312,0.068057,0.001758,0.051939,0.041590
4,pt,4,0.422123,0.376169,0.007735,0.363314,0.472040,0.034751,0.058782,-0.293545,...,0.009698,0.002776,0.102866,0.006474,0.000313,0.002511,0.001939,0.001885,0.027241,0.059010
5,pt,5,0.469541,0.422123,0.005408,0.424517,0.470169,0.032413,-0.278344,-0.226338,...,0.000938,0.000396,0.104659,0.058320,0.000906,0.054990,0.004882,0.000865,0.079380,0.033861
6,pt,6,0.521940,0.469541,0.007314,0.458595,0.466100,0.034591,0.158425,-0.339539,...,0.009474,0.004153,0.151505,0.120061,0.000815,0.077302,0.012009,0.001748,0.114769,0.011159
7,pt,7,0.585257,0.521940,0.007028,0.374758,0.432855,0.034730,-0.197557,-0.339422,...,0.105497,0.002020,0.125626,0.027422,0.000642,0.044215,0.012873,0.001254,0.079397,0.040308
8,pt,8,0.674587,0.585257,0.006375,0.367623,0.515404,0.031048,-0.278700,-0.198074,...,0.051463,0.004394,0.045512,0.118432,0.000420,0.000559,0.111236,0.001351,0.039484,0.043038
9,pt,9,1.193084,0.674587,0.004335,0.461843,0.526168,0.019339,-0.375559,-0.026030,...,0.079289,0.000680,0.046282,0.068083,0.000348,0.065936,0.057031,0.000782,0.041652,0.087165
